In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np

## Ensemble(Stacking) step

* The first trial would be to stacking model2 and model3

In [2]:
model2 = lgb.Booster(model_file='models/model_with_param2.lgb')
model3 = lgb.Booster(model_file='models/model_with_param3.lgb')
# model4 = lgb.Booster(model_file='model_6_7_14_28_fe.lgb')

### Data

In [6]:
## reading data
df_train_full =  pd.read_csv("sales_train_evaluation/sales_train_evaluation.csv")
df_calendar = pd.read_csv("calendar.csv")
df_prices = pd.read_csv("sell_prices.csv")
df_sample_submission = pd.read_csv("sample_submission.csv")
df_sample_submission["order"] = range(df_sample_submission.shape[0])

df_train = df_train_full.iloc[:, :-28]
df_valid = df_train_full.iloc[:, -28:]

In [3]:
#Preprocessed data
df_train_pre = pd.read_csv('preprocessed_df_7_28.csv')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\python demo\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-3debf8a819b7>", line 2, in <module>
    df_train_pre = pd.read_csv('preprocessed_df_7_28.csv')
  File "d:\python demo\lib\site-packages\pandas\io\parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "d:\python demo\lib\site-packages\pandas\io\parsers.py", line 463, in _read
    data = parser.read(nrows)
  File "d:\python demo\lib\site-packages\pandas\io\parsers.py", line 1169, in read
    df = DataFrame(col_dict, columns=columns, index=index)
  File "d:\python demo\lib\site-packages\pandas\core\frame.py", line 411, in __init__
    mgr = init_dict(data, index, columns, dtype=dtype)
  File "d:\python demo\lib\site-packages\pandas\core\internals\construction.py", line 257, in init_dict
    return arrays_to_mgr(arrays, data_names, index, columns, dt

KeyboardInterrupt: 

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df_train_pre.columns[~df_train_pre.columns.isin(useless_cols)]
X_train = df_train_pre[train_cols]
y_train = df_train_pre["sales"]

### 5-fold cross validation

In [ ]:
df_train_index = [ind for ind in df_train_pre.index]

In [47]:
k = 5
fold_size = len(df_train_index)/k

df_train_index = [ind for ind in df_train_pre.index]

params_3 = {
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'subsample': 0.5,
            'subsample_freq': 1,
            'learning_rate': 0.03,
            'num_leaves': 2**11-1,
            'min_data_in_leaf': 2**12-1,
            'feature_fraction': 0.5,
            'max_bin': 100,
            'n_estimators': 1400,
            'boost_from_average': False,
            'verbose': 1,
}

#5-fold validation loop
used_ind = []
for i in range(5):
    #This step is to avoid duplicate validation set
    used_indices = pd.Series(used_ind)
    total = pd.Series(df_train_index)
    valid_cand = total[~total.isin(used_indices)].to_list()
    ###############################################################################
    #First argument is all candidates of unselected index
    #Second argument is the size of each fold
    #Third argument is set False to ensure no element is reused
    validation_fold_index = np.random.choice(valid_cand,2,replace=False)
    training_folds_index = np.setdiff1d(df_train_index,validation_fold_index)
    #Add used index into this list
    used_ind.append(validation_fold_index)
    ###################################################################################
    lgb_train_data = lgb.Dataset(X_train.loc[training_folds_index] , label = y_train.loc[training_folds_index], 
                             categorical_feature=cat_feats, free_raw_data=False)
    lgb_valid_data = lgb.Dataset(y_train.loc[validation_fold_index], label = y_train.loc[validation_fold_index],
                                  categorical_feature=cat_feats,
                     free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!
    m_lgb = lgb.train(params_3, lgb_train_data, valid_sets = [lgb_valid_data], verbose_eval=20)
    m_lgb.save_model(f'model_ensemble_fold{i}.lgb')

array([2, 3])